# Preparing volumetric data for the atlas website

To upload volumetric based data to the website you will need to provide
- A `volume` array containing the volume to be displayed. The volume must be registered to the Allen CCF mouse atlas.
- The names of the volume feature.

Here we go into some steps that must be taken to upload your data to the website.

In all the examples below we will show how to upload the features to both a local and a remote bucket. To understand the difference between the two, please refer to the getting started tutorial (TODO LINK).

## Register your volume to the Allen CCF

The website currently only supports mouse volumes that have been registered and aligned to the Allen CCF mouse brain. A number of useful tools exist for this process

These include,
- [brainreg](https://brainglobe.info/index.html)
- [brainregister](https://github.com/int-brain-lab/brainregister)
- [HERBS](https://github.com/Whitlock-Group/HERBS/tree/main)


## Resample your volume to 25 um

The volume uploaded must have a resolution of 25 um in the ml, ap and dv dimensions. You can use the `simpleITK` python package to resample your data. This can be installed using `pip install SimpleITK`

The example below shows how you would resample a 10 um resolution volume to 25 um resolution.

In [ ]:
import SimpleITK as sitk
from pathlib import Path

# Path to original volume
img_path = Path.home().joinpath('path_to_volume', 'volume.tif')
save_img_path = img_path.parent.joinpath('volume_resampled.tif')

# Read in tiff image using simpleITK
reader = sitk.ImageFileReader()
reader.SetImageIO("TIFFImageIO")
reader.SetFileName(str(img_path))
img = reader.Execute()

# Resample the image
img_size = img.GetSize()
out_spacing = [2.5, 2.5, 2.5]
out_size = [int(img_size[0] / out_spacing[0]), int(img_size[1] / out_spacing[1]), int(img_size[2] / out_spacing[2])]
resample = sitk.ResampleImageFilter()
resample.SetOutputSpacing(out_spacing)
resample.SetSize(out_size)
resample.SetOutputDirection(img.GetDirection())
resample.SetOutputOrigin(img.GetOrigin())
resample.SetTransform(sitk.Transform())
resample.SetDefaultPixelValue(img.GetPixelIDValue())
resample.SetInterpolator(sitk.sitkBSpline)
new_img = resample.Execute(img)

# Save the image
# Write file to disk
writer = sitk.ImageFileWriter()
writer.SetImageIO("TIFFImageIO")
writer.SetFileName(str(save_img_path))
writer.Execute(new_img)

## Adjust the axis of your volume

The orientation of the volume should be [ap, dv, ml]